# Tutor-Student Assignment Problem --- Bipartite Matching with One Sided Preferences

The exact problem statement is redacted for privacy reasons. However, the problem can be summarized as follows:
- We have a set of tutors, each with a preferred centre and an upper bound on the number of students they can handle.
- A set of new students, and a set of existing students. All students are associated with a centre. Existing students are additionally associated with a tutor.
- Some students have constraints on which tutors they can be assigned to based on their tutoring needs.

Assumptions:
- For all students, changing of centres is not allowed.
- For existing students, changing of tutors is not allowed. Exceptions are granted for students that are matched wrongly based on tutoring needs.
- A feasible assignment exists, that is, the number of students does not exceed the total capacity of tutors, for both normal and extensive teaching needs.
- Tutors and students do not have preferences over each other, except for the tutoring needs constraints.
- Existing students that are inactive, do not affect the decision making.
- Tutors can teach at multiple centres.

The problem can be modeled as a bipartite graph where new students are to be assigned to tutors under certain objectives and constraints. The objectives and constraints will be described in detail in later sections.


## 1. Prepare imports and data

In [1]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model
import os
from IPython.display import display

# set wd
notebook_path = os.path.dirname(os.path.abspath("__file__"))
os.chdir(notebook_path)
# load data
myfile = 'Interview small data.xlsx'
new_students = pd.read_excel(open(myfile, 'rb'), sheet_name='New Students')
existing_students = pd.read_excel(open(myfile, 'rb'), sheet_name='Existing Students')
tutors = pd.read_excel(open(myfile, 'rb'), sheet_name='Tutor Information')

ModuleNotFoundError: No module named 'pandas'

## 2. Data validation

In [ ]:
# view data
print("Data preview:")
print("New students:")
display(new_students.head())
print("Existing students:")
display(existing_students.head())
print("Tutors:")
display(tutors.head())

# comments
print("Note that new students are assigned smaller IDs than existing students.\nThis means that existing students will always have their IDs changed/incremented every time new students are added.")


### 2.1. Quick validatation

In [ ]:
# remove existing students who are inactive
print("Inactive existing students will be removed.\n")
existing_students = existing_students[existing_students['active']==1]

# check missing data
print("Missing data check:")
print("new students: ", new_students.isnull().sum().sum())
print("existing students: ", existing_students.isnull().sum().sum())
print("tutors: ", tutors.isnull().sum().sum())
print("\n")
# check duplicates
print("Duplicates check:")
print("new students:", new_students.duplicated().sum())
print("existing students:", existing_students.duplicated().sum())
print("tutors:", tutors.duplicated().sum())
print("\n")

# check that the tutor of existing students exists in tutors
print("Existing students' tutor check:")
tutor_ids = set(tutors['tutorId'])
existing_student_tutor_ids = set(existing_students['tutorId'])
if existing_student_tutor_ids.issubset(tutor_ids):
    print("All existing students have valid tutors.")
else:
    print("Warning: Some existing students have tutors not listed in the tutors data.")
print("\n")

# check sum of max capacity of tutors >= number of new students + existing active students
print("Capacity check:")
numtut = tutors.shape[0]
tutcap = tutors['maxOverallCapacity'].sum()
excap = existing_students.shape[0]
newcap = new_students.shape[0]
print("number of tutors:", numtut)
print("tutors capacity:", tutcap)
print("existing students count:", excap)
print("new students count:", newcap)
if tutcap >= excap + newcap:
    print("Capacity is sufficient.")
else:
    print("No feasible solution possible: insufficient capacity. Please check data.")
print("\n")

# tutor capacity check
print("Tutor individual capacity check (based on existing students):")
tutor_capacity = tutors.set_index('tutorId')['maxOverallCapacity'].to_dict()
existing_student_count = existing_students.groupby('tutorId').size().to_dict()
ok = True
for tutor, count in existing_student_count.items():
    if count > tutor_capacity[tutor]:
        print(f"Warning: Tutor {tutor} is over capacity with {count} existing students.")
        ok = False
if ok:
    print("All tutors (individually) are within their capacity limits, based on existing students.")

### 2.2. Further validation based on tutoring needs

In [ ]:
# check that existing students are matched to a tutor that meets their tutoring needs
existing_students_needs = existing_students.merge(tutors[['tutorId', 'tutoringSkills']], on='tutorId', how='left')
mismatch = existing_students_needs[~existing_students_needs.apply(lambda row: row['tutoringNeed'] == row['tutoringSkills'], axis=1)]
print("Existing students' tutoring needs check:")
print(f"Number of existing students who require extensive tutoring but are matched with a normal tutoringSkills tutor: {mismatch.shape[0]}")
print(f"{mismatch.shape[0]} existing students will be treated as new students for the purpose of assignment.")
# remove these students from existing students to new students
existing_students = existing_students[existing_students['studentId'].isin(mismatch['studentId'])==False]
new_students = pd.concat([new_students, mismatch.drop(columns=['active', 'tutorId', 'tutoringSkills'])], ignore_index=True)
print("\n")

# check that no "extensive" tutor is overloaded
print("Extensive tutor capacity check (based on existing and new students):")
ok = True
# individual check
extensive_tutors = tutors[tutors['tutoringSkills'] == 'Extensive']
for tutor in extensive_tutors['tutorId']:
    assigned_students = existing_students[existing_students['tutorId'] == tutor]
    if assigned_students.shape[0] > extensive_tutors[extensive_tutors['tutorId'] == tutor]['maxOverallCapacity'].values[0]:
        print(f"Warning: Tutor {tutor} is overloaded with {assigned_students.shape[0]} students.")
        ok = False
# collective check
extensive_tutor_capacity = extensive_tutors['maxOverallCapacity'].sum()
existing_extensive_students = existing_students[existing_students['tutoringNeed'] == 'Extensive'].shape[0]
new_extensive_students = new_students[new_students['tutoringNeed'] == 'Extensive'].shape[0]
if extensive_tutor_capacity < existing_extensive_students + new_extensive_students:
    print("No feasible solution possible: insufficient extensive tutor capacity. Please check data.")
    ok = False
if ok:
    print("All extensive tutors (individually and collectively) are within their capacity limits, based on existing and new students.")

## 3. Mathematical formulation

We define the following mathematical formulation for the problem.

Parameters:
- Let $E$ be the set of existing students.
- Let $N$ be the set of new students.
- Let $T$ be the set of tutors.
- Let $T_{ext}$ be the set of tutors that can handle extensive tutoring needs.
- Let $N_{ext}$ be the set of new students that require extensive tutoring needs.
- Let $cap_t$ be the capacity of tutor $t$.
- Let $y_{te}$ be a constant that is 1 if tutor $t$ was assigned to existing student $e$, and 0 otherwise.
- Let $c_{tn}$ be the coefficient for tutor $t$ to be assigned to new student $n$, based on the student's centre and tutor's preferred centre. We set $c_{tn} = 2$ if it is the tutor's first choice of centre, $c_{tn} = 1$ if it is the tutor's second choice of centre, and $c_{tn} = 0$ otherwise.

Variables:
- Let $x_{tn}$ be a binary variable that is 1 if tutor $t$ is assigned to new student $n$, and 0 otherwise.

Constraints:
- Each new student is assigned to exactly one tutor:
   $$\sum_{t \in T} x_{tn} = 1 \quad \forall n \in N$$
- Each new student with extensive tutoring needs is assigned to a tutor that can handle extensive tutoring needs:
   $$\sum_{t \in T_{ext}} x_{tn} = 1 \quad \forall n \in N_{ext}$$
- A tutor cannot exceed their maximum overall capacity:
   $$\sum_{n \in N} x_{tn} + \sum_{e \in E} y_{te} \leq cap_t \quad \forall t \in T$$

### 3.1. Task (a)

Problem statement:
Minimize total number of tutors assigned while maximizing tutor’s preference on tuition centre.

Objectives:
- Minimize total number of tutors assigned to at least one new student or existing student:
    $$\min \sum_{t \in T} z_t, $$
    where $z_t$ is a binary variable that is 1 if tutor $t$ is assigned to at least one new student or existing student, and 0 otherwise.
- Maximize tutor's preference on tuition centre (for new students):
    $$\max \sum_{t \in T} \sum_{n \in N} c_{tn} x_{tn}.$$
Objective function:
- Combine the two objectives into a single objective function:
    $$\min \sum_{t \in T} z_t - \alpha \sum_{t \in T} \sum_{n \in N} c_{tn} x_{tn},$$
    where $\alpha$ is a weighting factor to balance the two objectives. Range of values for first objective is $[0, |T|]$, while range of values for second objective is $[0, 2|S|]$. We set $\alpha = \frac{|T|}{2|S|}$ to balance the two objectives. Here, we set $\alpha = \frac{10}{64} = 0.15625$.

Additional constraints:
- Link $z_t$ with $x_{tn}$ and $y_{te}$:
   $$M z_t \geq \sum_{n \in N} x_{tn} + \sum_{e \in E} y_{te} \quad \forall t \in T,$$
   where $M$ is a sufficiently large constant, which we set to a value greater the sum of all tutor capacities (57). Here, we set $M = 100$.

#### 3.1. Code for task (a)

##### 3.1.1. Prepare data for model

In [ ]:
# create copy of tables we will need, and add column of index
E = existing_students.copy().reset_index(drop=True).rename_axis('e').reset_index()
N = new_students.copy().reset_index(drop=True).rename_axis('n').reset_index()
T = tutors.copy().reset_index(drop=True).rename_axis('t').reset_index()

# create minitable of centres and index
ctrs = pd.DataFrame(pd.concat([new_students['tuitionCentre'], existing_students['tuitionCentre']]).unique(), columns=['tuitionCentre'])
ctrs = ctrs.reset_index().rename(columns={'index': 'c'})

# ctn: coefficient for tutor t to be assigned to new student n
# also add col to mark a compatible student and tutor where both are extensive
ctn = pd.merge(pd.merge(T.assign(key=1), N.assign(key=1), on='key').drop('key', axis=1),
               ctrs.assign(key=1), on='tuitionCentre', how='left')
ctn['c_tn'] = np.where(ctn['preferredCentre1'] == ctn['tuitionCentre'], 2,
                       np.where(ctn['preferredCentre2'] == ctn['tuitionCentre'], 1, 0))
ctn['both_ext'] = np.where((ctn['tutoringSkills'] == 'Extensive') & (ctn['tutoringNeed'] == 'Extensive'), 1, 0)
ctn = ctn[['t', 'n', 'c_tn', 'both_ext']]
both_ext_dict = {(row['t'], row['n']): 1 for _, row in ctn.iterrows() if row['both_ext'] == 1}
c_tn_dict = ctn.set_index(["t", "n"])["c_tn"].to_dict()

# check some values
print("ctn preview:")
print("Both extensive:")
display(ctn[ctn['both_ext'] == 1].head())
print("\n")
print("c_tn > 0 (student at centre that tutor likes):")
display(ctn[ctn['c_tn'] > 0].head())

#### 3.1.2. Build and solve model

Constants/variables:

In [ ]:
model_a = Model(name="model_a")
# constants
ALPHA = 0.15625
M = 100
# variables
x = model_a.binary_var_dict(
    [(row["t"], row["n"]) for _, row in ctn.iterrows()], name="x"
)
y = {(row["t"], row["e"]): 1 for _, row in pd.merge(T, E, on="tutorId").iterrows()}
z = model_a.binary_var_dict([(row["t"]) for _, row in T.iterrows()], name="z")

Constraints:

In [ ]:
# each new student is assigned to exactly one tutor
for n in N["n"]:
    model_a.add_constraint(
        model_a.sum(x[t, n] for t in T["t"]) == 1, ctname=f"assign_new_student_{n}"
    )

# each new student that requires extensive tutoring is assigned to an extensive tutor
for n in N[N["tutoringNeed"] == "Extensive"]["n"]:
    model_a.add_constraint(
        model_a.sum(x[t, n] * both_ext_dict.get((t, n), 0) for t in T["t"]) == 1,
        ctname=f"assign_extensive_new_student_{n}",
    )

# each tutor does not exceed their capacity
for _, row in T.iterrows():
    t = row["t"]
    cap_t = row["maxOverallCapacity"]
    expr_n = (
        model_a.sum(x[t, n] for n in N["n"] if (t, n) in x) or 0
    )  # new students assigned
    expr_e = sum(y.get((t, e), 0) for e in E["e"])  # fixed existing students
    model_a.add_constraint(expr_n + expr_e <= cap_t, ctname=f"tutor_capacity_{t}")

# additional constraint: link z_t with x_tn and y_te
for _, row in T.iterrows():
    t = row["t"]
    expr_n = (
        model_a.sum(x[t, n] for n in N["n"] if (t, n) in x) or 0
    )  # new students assigned
    expr_e = sum(y.get((t, e), 0) for e in E["e"])  # fixed existing students
    model_a.add_constraint(M * z[t] >= expr_n + expr_e, ctname=f"link_z_{t}")

Objective function and model solve:

In [ ]:
model_a.minimize(
    model_a.sum(z[t] for t in T["t"])
    - ALPHA * model_a.sum(x[t, n] * c_tn_dict.get((t, n), 0) for t, n in x)
)

print("Objective expression:", model_a.objective_expr) # always good to check
# seems correct. i see that -0.312x_0_0 means tutor 0 assigned to new student 0, who is at tutor 0's preferredCentre1 since 2*0.15625 = 0.3125
# and there is no x_0_1 since student 1 wants west, which is neither of tutor 0's preferred centres (coeff = 0)
# there is all the z_t terms with coeff 1, which is correct

# solve
# note: we could make use of functions to modularize the code better
# but only (a) and (b) are required - at most repeat once. it's ok for today
solution = model_a.solve()
if solution:
    print("Solution status:", model_a.get_solve_status())
    # print("Objective value:", model_a.objective_value) # doesn't explain much
    # extract solution
    assigned = []
    for (t, n), var in x.items():
        if var.solution_value == 1:
            assigned.append((t, n))
    assigned_df = pd.DataFrame(assigned, columns=["t", "n"])
    assigned_df = assigned_df.merge(T[["tutorId", "t"]], on="t", how="left").merge(N[["studentId", "n"]], on="n", how="left")
    print("Assigned tutor-student pairs:")
    display(assigned_df)
else:
    print("No solution found.")
    
# from dataframe we see that assigned_df['n'] is a permutation of all new students' n
# i.e. all new students are assigned to exactly one tutor

Analysis of results:

In [ ]:
# how many students assigned to each tutor, including existing students
print("Summary of number of students assigned to each tutor (including existing students):")
tutor_assignments = assigned_df.groupby('tutorId').size().reset_index(name='num_assigned_students')
tutor_assignments = tutor_assignments.merge(tutors[['tutorId', 'maxOverallCapacity']], on='tutorId', how='left')
existing_df = existing_students.groupby('tutorId').size().reset_index(name='num_existing_students')
# forgot to put put tutor capacity for existing students!!
existing_df = existing_df.merge(tutors[['tutorId', 'maxOverallCapacity']], on='tutorId', how='left')
tutor_summary = pd.merge(tutor_assignments, existing_df, on='tutorId', how='outer').fillna(0)
# tutor capacity should be the max of both columns
tutor_summary['maxOverallCapacity'] = tutor_summary[['maxOverallCapacity_x', 'maxOverallCapacity_y']].max(axis=1)
# remove the old capacity cols
tutor_summary = tutor_summary.drop(columns=['maxOverallCapacity_x', 'maxOverallCapacity_y'])
tutor_summary['total_students'] = tutor_summary['num_assigned_students'] + tutor_summary['num_existing_students']
tutor_summary['total_students'] = tutor_summary['total_students'].astype(int) # convert to int
tutor_summary = tutor_summary[['tutorId', 'total_students', 'maxOverallCapacity']]
display(tutor_summary)
print(f"We have created assignments for {tutor_summary.shape[0]} out of {tutors.shape[0]} tutors.\n")

# add 3 columns to count how many pref1, pref2, non-pref students assigned to each tutor
assigned_df_2 = assigned_df.merge(ctn[['t', 'n', 'c_tn']], on=['t', 'n'], how='left').copy()
pref1_count = assigned_df_2[assigned_df_2['c_tn'] == 2].groupby('tutorId').size().reset_index(name='num_pref1_new')
pref2_count = assigned_df_2[assigned_df_2['c_tn'] == 1].groupby('tutorId').size().reset_index(name='num_pref2_new')
nonpref_count = assigned_df_2[assigned_df_2['c_tn'] == 0].groupby('tutorId').size().reset_index(name='num_nonpref_new')
tutor_summary = tutor_summary.merge(pref1_count, on='tutorId', how='left').merge(pref2_count, on='tutorId', how='left').merge(nonpref_count, on='tutorId', how='left').fillna(0)
tutor_summary['num_pref1_new'] = tutor_summary['num_pref1_new'].astype(int)
tutor_summary['num_pref2_new'] = tutor_summary['num_pref2_new'].astype(int)
tutor_summary['num_nonpref_new'] = tutor_summary['num_nonpref_new'].astype(int)
tutor_summary = tutor_summary[['tutorId', 'num_pref1_new', 'num_pref2_new', 'num_nonpref_new']]
print("Number of new students assigned based on tutor's preference:")
display(tutor_summary)
print("Tutors are generally assigned to students at their preferred centres. No tutor was assigned a non-preferred centre.\n")

### 3.2. Task (b)

Problem statement: Balance tutor’s workload while maximizing tutor’s preference on the tuition centre. 

Objectives:
- Minimize the maximum number of students assigned to any tutor:
    $$\min \max_{t \in T} \left( \sum_{n \in N} x_{tn} + \sum_{e \in E} y_{te} \right) $$
- Maximize tutor's preference on tuition centre (for new students):
    $$\max \sum_{t \in T} \sum_{n \in N} c_{tn} x_{tn}.$$
Objective function:
- Combine the two objectives into a single objective function:
    $$\min \max_{t \in T} \left( \sum_{n \in N} x_{tn} + \sum_{e in E} y_{te} \right) - \beta \sum_{t \in T} \sum_{n \in N} c_{tn} x_{tn},$$
    where $\beta$ is a weighting factor to balance the two objectives. Range of values for first objective is $[0, \max_{t \in T} cap_t]$, while range of values for second objective is $[0, 2|S|]$. We set $\beta = \frac{\max_{t \in T} cap_t}{2|S|}$ to balance the two objectives. Here, we set $\beta = \frac{8}{64} = 0.125$.
    

#### 3.2. Code for task (b)

##### 3.2.1. Prepare data for model
Quick recap of defined items from (a):
- E: dataframe of existing students
- N: dataframe of new students
- T: dataframe of tutors
- ctrs: enumerated mapping int to centre name
- ctn: dictionary mapping (tutor, student) pair to coefficient based on tutor's preferred centres
- both_ext_dict: dictionary mapping student to whether they require extensive tutoring needs
- c_tn_dict: dictionary mapping (tutor, student) pair to whether tutor can handle student's extensive tutoring needs

#### 3.2.2. Build and solve model

Constants/variables:

In [ ]:
model_b = Model(name="model_b")
# constants
BETA = 0.125
# variables
x = model_b.binary_var_dict(
    [(row["t"], row["n"]) for _, row in ctn.iterrows()], name="x"
)
y = {(row["t"], row["e"]): 1 for _, row in pd.merge(T, E, on="tutorId").iterrows()}

Constraints:

In [ ]:
# each new student is assigned to exactly one tutor
for n in N["n"]:
    model_b.add_constraint(
        model_b.sum(x[t, n] for t in T["t"]) == 1, ctname=f"assign_new_student_{n}"
    )
# each new student that requires extensive tutoring is assigned to an extensive tutor
for n in N[N["tutoringNeed"] == "Extensive"]["n"]:
    model_b.add_constraint(
        model_b.sum(x[t, n] * both_ext_dict.get((t, n), 0) for t in T["t"]) == 1,
        ctname=f"assign_extensive_new_student_{n}",
    )
# each tutor does not exceed their capacity
for _, row in T.iterrows():
    t = row["t"]
    cap_t = row["maxOverallCapacity"]
    expr_n = (
        model_b.sum(x[t, n] for n in N["n"] if (t, n) in x) or 0
    )  # new students assigned
    expr_e = sum(y.get((t, e), 0) for e in E["e"])  # fixed existing students
    model_b.add_constraint(expr_n + expr_e <= cap_t, ctname=f"tutor_capacity_{t}")

Objective function and model solve:

In [ ]:
model_b.minimize(
    model_b.max(
        model_b.sum(x[t, n] for n in N["n"]) + sum(y.get((t, e), 0) for e in E["e"])
        for t in T["t"]
    )
    - BETA * model_b.sum(x[t, n] * c_tn_dict.get((t, n), 0) for t, n in x)
)

print("Objective expression:", model_b.objective_expr) # always good to check
# the 2nd part is the same as model_a, so looks good
# the 1st part of max looks correct too. i see max(x_0_0, x_0_1 ...x_0_n, x1_0, x_1_1,....)
# also i saw for tutor 4 (base 0), there was a +4 behind the max, and tutor 4 (A005) had 4 existing students (normal tutoringNeed)

# solve
solution_b = model_b.solve()
if solution_b:
    print("Solution status:", model_b.get_solve_status())
    # print("Objective value:", model_b.objective_value) # doesn't explain much
    # extract solution
    assigned_b = []
    for (t, n), var in x.items():
        if var.solution_value == 1:
            assigned_b.append((t, n))
    assigned_df_b = pd.DataFrame(assigned_b, columns=["t", "n"])
    assigned_df_b = assigned_df_b.merge(T[["tutorId", "t"]], on="t", how="left").merge(
        N[["studentId", "n"]], on="n", how="left"
    )
    print("Assigned tutor-student pairs:")
    display(assigned_df_b)
else:
    print("No solution found.")

Analysis of results:

In [ ]:
# quick sanity check. extensive needs students are assigned to extensive tutors
extensive_assignments = assigned_df_b.merge(N[['studentId', 'tutoringNeed']], on='studentId', how='left').merge(T[['tutorId', 'tutoringSkills']], on='tutorId', how='left')
extensive_mismatch = extensive_assignments[(extensive_assignments['tutoringNeed'] == 'Extensive') & (extensive_assignments['tutoringSkills'] != 'Extensive')]
print(f"Number of extensive-need new students assigned to non-extensive tutors: {extensive_mismatch.shape[0]}\n")

# how many students assigned to each tutor, including existing students
print("Summary of number of students assigned to each tutor (including existing students):")
tutor_assignments = assigned_df_b.groupby('tutorId').size().reset_index(name='num_assigned_students')
tutor_assignments = tutor_assignments.merge(tutors[['tutorId', 'maxOverallCapacity']], on='tutorId', how='left')
existing_df = existing_students.groupby('tutorId').size().reset_index(name='num_existing_students')
# forgot to put put tutor capacity for existing students!!
existing_df = existing_df.merge(tutors[['tutorId', 'maxOverallCapacity']], on='tutorId', how='left')
tutor_summary = pd.merge(tutor_assignments, existing_df, on='tutorId', how='outer').fillna(0)
# tutor capacity should be the max of both columns
tutor_summary['maxOverallCapacity'] = tutor_summary[['maxOverallCapacity_x', 'maxOverallCapacity_y']].max(axis=1)
# remove the old capacity cols
tutor_summary = tutor_summary.drop(columns=['maxOverallCapacity_x', 'maxOverallCapacity_y'])
tutor_summary['total_students'] = tutor_summary['num_assigned_students'] + tutor_summary['num_existing_students']
tutor_summary['total_students'] = tutor_summary['total_students'].astype(int) # convert to int
tutor_summary['maxOverallCapacity'] = tutor_summary['maxOverallCapacity'].astype(int) # convert to int
tutor_summary = tutor_summary[['tutorId', 'total_students', 'maxOverallCapacity']]
display(tutor_summary)
print(f"Looks good. The worst-off tutor has {tutor_summary['total_students'].max()} students assigned. We have distributed the assignments among {tutor_summary.shape[0]} out of {tutors.shape[0]} tutors.\n")

# add 3 columns to count how many pref1, pref2, non-pref students assigned to each tutor
assigned_df_2 = assigned_df_b.merge(ctn[['t', 'n', 'c_tn']], on=['t', 'n'], how='left').copy()
pref1_count = assigned_df_2[assigned_df_2['c_tn'] == 2].groupby('tutorId').size().reset_index(name='num_pref1_new')
pref2_count = assigned_df_2[assigned_df_2['c_tn'] == 1].groupby('tutorId').size().reset_index(name='num_pref2_new')
nonpref_count = assigned_df_2[assigned_df_2['c_tn'] == 0].groupby('tutorId').size().reset_index(name='num_nonpref_new')
tutor_summary = tutor_summary.merge(pref1_count, on='tutorId', how='left').merge(pref2_count, on='tutorId', how='left').merge(nonpref_count, on='tutorId', how='left').fillna(0)
tutor_summary['num_pref1_new'] = tutor_summary['num_pref1_new'].astype(int)
tutor_summary['num_pref2_new'] = tutor_summary['num_pref2_new'].astype(int)
tutor_summary['num_nonpref_new'] = tutor_summary['num_nonpref_new'].astype(int)
tutor_summary = tutor_summary[['tutorId', 'num_pref1_new', 'num_pref2_new', 'num_nonpref_new']]
print("Number of new students assigned based on tutor's preference:")   
display(tutor_summary)
print("Tutors are generally assigned to students at their preferred centres.\nOnly 2 tutors were assigned to a single student at a pref2 centre. No tutor was assigned a non-preferred centre.\n")